In [ ]:
import os
import pickle
import numpy
import random 

from music21 import note, chord
from models.RNNAttention import get_distinct, create_lookups, prepare_sequences, get_music_list, create_network
from keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.utils import plot_model

In [ ]:
from music21 import corpus, converter
from IPython.display import Image, Audio
def play(music):
  filename = music.write('mid')
  !fluidsynth -ni font.sf2 $filename -F $filename\.wav -r 16000 > /dev/null
  display(Audio(str(filename) + '.wav'))

In [ ]:
BASE_DIR = os.getcwd()
music_folder = 'classical' 
music_name = 'classical'

run_folder = os.path.join(BASE_DIR,"output",music_folder)
store_folder = os.path.join(run_folder, 'store')
data_folder = os.path.join(BASE_DIR,'data', music_name)

if not os.path.exists(run_folder):
    os.makedirs(run_folder)
    os.makedirs(os.path.join(run_folder, 'store'))
    os.makedirs(os.path.join(run_folder, 'output'))
    os.makedirs(os.path.join(run_folder, 'weights'))
    os.makedirs(os.path.join(run_folder, 'viz'))
    
mode = 'load' # 'load' # 

# data params
intervals = range(1)
seq_len = 32

# model params
embed_size = 100
rnn_units = 256
use_attention = True

In [ ]:
import random
#Tranforma los datos a un formato más estructurado, separando la pista en los silencios, los acordes, las notas, etc.
songs = []
if mode == 'build':
    
    music_list, parser = get_music_list(data_folder)
    print(len(music_list), 'files in total')

    notes = []
    durations = []
    test = 1
    for i in random.sample(range(0, len(music_list)), 125):
        songs.append(os.path.basename(music_list[i]))
        try:
            if test%5==0:
                print('-'*20,test,'-'*20)
            #Desconpone la pista en acordes 
            original_score = parser.parse(music_list[i]).chordify()
            
            for interval in intervals:
                score = original_score.transpose(interval)
                notes.extend(['START'] * seq_len)
                durations.extend([0]* seq_len)
                #Se verifica que tipo de entrada es (silencio, acorde, nota) y se almacena junto con su duration en dos lista (notes, durations)
                for element in score.flat:
                    
                    if isinstance(element, note.Note):
                        if element.isRest:
                            notes.append(str(element.name))
                            durations.append(element.duration.quarterLength)
                        else:
                            notes.append(str(element.nameWithOctave))
                            durations.append(element.duration.quarterLength)

                    if isinstance(element, chord.Chord):
                        notes.append('.'.join(n.nameWithOctave for n in element.pitches))
                        durations.append(element.duration.quarterLength)
            test+=1
        except:
            print('error in file: %s' % file)
            os.remove(file)
    #Crearemos o sobreescribiremos los archivos donde se almacenaras las notas y duraciones
    with open(os.path.join(store_folder, 'notes'), 'wb') as f:
        pickle.dump(notes, f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, 'durations'), 'wb') as f:
        pickle.dump(durations, f) 
    with open(os.path.join(store_folder, 'train.txt'), 'w') as f:
        for s in songs:
            f.write(str(s) + '\n')
            
#En el caso de que no nos encontremos en modo "build", se cargarán los datos previamente almacenados
else:
    with open(os.path.join(store_folder, 'notes'), 'rb') as f:
        notes = pickle.load(f) #['G2', 'D3', 'B3', 'A3', 'B3', 'D3', 'B3', 'D3', 'G2',...]
    with open(os.path.join(store_folder, 'durations'), 'rb') as f:
        durations = pickle.load(f)

In [ ]:
note_names, n_notes = get_distinct(notes)
duration_names, n_durations = get_distinct(durations)
distincts = [note_names, n_notes, duration_names, n_durations]

with open(os.path.join(store_folder, 'distincts'), 'wb') as f:
    pickle.dump(distincts, f)

note_to_int, int_to_note = create_lookups(note_names)
duration_to_int, int_to_duration = create_lookups(duration_names)
lookups = [note_to_int, int_to_note, duration_to_int, int_to_duration]

with open(os.path.join(store_folder, 'lookups'), 'wb') as f:
    pickle.dump(lookups, f)

In [ ]:
network_input, network_output = prepare_sequences(notes, durations, lookups, distincts, seq_len)

In [ ]:
model, att_model = create_network(n_notes, n_durations, embed_size, rnn_units, use_attention)

In [ ]:
weights_folder = os.path.join(run_folder, 'weights')

checkpoint1 = ModelCheckpoint(
    os.path.join(weights_folder, "weights-improvement-{epoch:02d}-{loss:.4f}-bigger.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

checkpoint2 = ModelCheckpoint(
    os.path.join(weights_folder, "weights.h5"),
    monitor='loss',
    verbose=0,
    save_best_only=True,
    mode='min'
)

early_stopping = EarlyStopping(
    monitor='loss'
    , restore_best_weights=True
    , patience = 10
)


callbacks_list = [
    checkpoint1,
    checkpoint2
   #,early_stopping
 ]

model.save_weights(os.path.join(weights_folder, "weights.h5"))
model.fit(network_input, network_output
          , epochs=150, batch_size=32
          , validation_split = 0.2
          , callbacks=callbacks_list
          , shuffle=True
         )